In [1]:
from _functions import *

In [ ]:

params = Params()

# 细分：融券户交易的参数，此时交易时点为日内连续竞价而不是尾盘
open_params = Params()
open_params.pt = Params()
open_params.pt.trade_type = 'PT' # 确定交易类型为普通买入卖出，此处不能更改
open_params.pt.amount_prop_sh = 0 # sh任务金额占总任务金额比例
open_params.pt.amount_prop_sz = 100 # sz任务金额占总任务金额比例
open_params.pt.max_prop_single = 4.0 # 单只股票单次最大比例，单位：%，沪深统一，而且是相对于当日买入金额
open_params.pt.max_prop_total = 4.0 # 单只股票总共最大比例，单位：%，沪深统一，而且是相对于合计买入金额
open_params.pt.trade_retmax = 9 # 收益率上限，单位：%
open_params.pt.trade_retmin =-9 # 收益率下限，单位：%
open_params.pt.buy_time = '930_940'
open_params.pt.sell_time = '1000_1430'
open_params.pt.trade_mode= 'vwap'
open_params.pt.tvr_limit = 70.0 # 换手率限制
open_params.pt.liquid_ratio = 3
open_params.pt.buy_prop_normal = 20.0

open_params.rq = Params()
open_params.rq.trade_type = 'RQ' # 确定交易类型为融券，此处不能更改
open_params.rq.rq_broker = 'GuangFaChangWai'
open_params.rq.pc_limit = 0 # 是否有低于昨收价限制，取决于用什么票池，1/0代表 普通融券/dma
open_params.rq.amount_prop_sh = 0 # sh任务金额占总任务金额比例
open_params.rq.amount_prop_sz = 100 # sz任务金额占总任务金额比例
open_params.rq.max_prop_single = 4.0 # 单只股票单次最大比例，单位：%，沪深统一，而且是相对于当日买入金额
open_params.rq.max_prop_total = 4.0 # 单只股票总共最大比例，单位：%，沪深统一，而且是相对于合计买入金额
open_params.rq.trade_retmax = 9 # 收益率上限，单位：%
open_params.rq.trade_retmin =-9 # 收益率下限，单位：%
open_params.rq.buy_time = '930_940'
open_params.rq.sell_time = '1000_1430'
open_params.rq.trade_mode= 'vwap'
open_params.rq.tvr_limit = 30.0 # 换手率限制
open_params.rq.liquid_ratio = 3
open_params.rq.sell_prop_normal = 15.0 

# 将参数放到全局变量中
params.open_params = open_params

class limit_params:
    citic_share_limit_stage1 = 0.05
    cmvg_share_limit_stage1 = 0.075
    citic_share_limit_stage2 = 0.05
    cmvg_share_limit_stage2 = 0.075

params.limit_params = limit_params

params.score_buy_adj = 0.5
params.score_sell_adj = 0.5

In [ ]:
def get_can_trade_amount_ot(stock_list, score_array, score_array_sell, params):


    # 记录这一时刻之前已经发生的多头账户买入和空头账户卖出的交易记录，（对于925时点，此时并不会发生其他交易发生）
    # 多头账户卖出和空头账户还券不需要记录
    # 原则上，今天已经买入的多头股票，不能再进行空头融券卖出，已经融券卖出的空头融券，不能再进行多头账户买入
    if len(params.today_open_trade_task) > 0:
        today_buy_stock_list = list(params.today_open_trade_task.loc[(params.today_open_trade_task.side == "B") & (params.today_open_trade_task.a_trans > 0) & (params.today_open_trade_task.type == "PT"), "code"].unique())
        today_sell_stock_list = list(params.today_open_trade_task.loc[(params.today_open_trade_task.side == "S") & (params.today_open_trade_task.a_trans > 0) & (params.today_open_trade_task.type == "RQ"), "code"].unique())
    else:
        today_buy_stock_list = []
        today_sell_stock_list = []

    # 同样获取当日持仓
    # 获取当前持仓股票的volume, 框架里面rq_volume全都是负数，-100代表融券100股
    stock_hold_pt = {code:round(stock_info.pt_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.pt_volume > 0}
    stock_available_pt = {code:round(stock_info.pt_available_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.pt_volume > 0}
    stock_hold_rq = {code:round(stock_info.rq_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.rq_volume < 0}
    stock_available_rq = {code:round(stock_info.rq_available_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.rq_volume < 0}

    # 获取昨收价格、涨跌停价格、融券余量、实时价格，都是您那边直接获取
    today_preclose_dict = params.stock_daily_data_set[params.date]['preclose'] # 昨收价
    today_ztprice_dict = params.stock_daily_data_set[params.date]['ztprice'] # 涨停价
    today_dtprice_dict = params.stock_daily_data_set[params.date]['dtprice'] # 跌停价
    today_open_dict = params.stock_daily_data_set[params.date]['open'] # 开盘价
    today_trade_price_dict = params.stock_trade_price_data_set[params.date] # 交易价格
    today_stock_trade_price_rolling_dict = params.stock_trade_price_rolling_data_set[params.date] # 由本地传入
    today_realtime_rq_dict = params.stock_realtime_rq_data_set[params.date] # 融券余量

    stock_buy_pt_liquid = {} # 多头账户可买入的流动性
    stock_buy_rq_liquid = {} # 空头账户可还券的流动性
    stock_sell_pt_liquid = {} # 多头账户可卖出的流动性
    stock_sell_rq_liquid = {} # 空头账户可融券卖出的流动性
    stock_sell_liquid ={}
    stock_buy_liquid ={}
    stock_price_now = {}

    for stock in stock_list:
        # 昨收价
        if stock not in today_preclose_dict.keys():
            continue
        pc = today_preclose_dict[stock] 
        if pc == 0.0 or pc != pc: # 如果昨收价为0或者为空值
                continue

        # 涨跌停限制
        p0 = today_open_dict[stock] # 获取当前股票价格p0，对于此时价格为开盘价
        if p0 == 0.0 or p0 != p0: # 如果当前价为0或者为空值
            stock_price_now[stock] = pc
            stock_buy_pt_liquid[stock] = 0
            stock_buy_rq_liquid[stock] = 0
            stock_sell_pt_liquid[stock] = 0
            stock_sell_rq_liquid[stock] = 0
            continue

        # 涨跌停对交易的限制
        stock_price_now[stock] = p0
        zp = today_ztprice_dict[stock] # 涨停价
        dp = today_dtprice_dict[stock] # 跌停价
        hp = my_round2(pc + (zp - pc) * params.open_params.pt.trade_retmax / 10) # 风控最高价
        lp = my_round2(pc + (pc - dp) * params.open_params.pt.trade_retmin / 10) # 风控最低价
        if p0 >= hp:
            stock_buy_pt_liquid[stock] = 0
            stock_buy_rq_liquid[stock] = 0
            stock_sell_pt_liquid[stock] = 0
            stock_sell_rq_liquid[stock] = 0
        if p0 <= lp:
            stock_buy_pt_liquid[stock] = 0
            stock_buy_rq_liquid[stock] = 0
            stock_sell_pt_liquid[stock] = 0
            stock_sell_rq_liquid[stock] = 0


        # 获取这个时点预估成交金额，此处由我传入，和930-940多头公用一个文件
        es_amt_buy = today_stock_trade_price_rolling_dict[stock][rf'asum_{params.open_params.pt.buy_time}_5d_mean'].fillna(0).iloc[0]
        es_amt_sell = today_stock_trade_price_rolling_dict[stock][rf'asum_{params.open_params.rq.sell_time}_5d_mean'].fillna(0).iloc[0]

        # 这一部分代表盘口流动性限制
        stock_buy_liquid[stock] = int(es_amt_buy * (params.open_params.pt.liquid_ratio / 100.0)) + 1
        stock_sell_liquid[stock] = int(es_amt_sell * (params.open_params.rq.liquid_ratio / 100.0)) + 1

        # 获取当前各个仓位，并转化为正数，此时v_pt_avai、v_rq_avai、v_pt_hold、v_rq_hold均大于等于0
        v_pt_avai = stock_available_pt.get(stock, 0) # 普通户可以卖出的持仓
        v_rq_avai =  - stock_available_rq.get(stock, 0) # 融券户可以买入的持仓
        v_pt_hold = stock_hold_pt.get(stock, 0)
        v_rq_hold = - stock_hold_rq.get(stock, 0)


        if v_pt_avai > 0:
            stock_sell_pt_liquid[stock] = min(stock_sell_liquid[stock],  int(v_pt_avai * p0)  + 1)
        else:
            stock_sell_pt_liquid[stock] = 0
        if v_rq_avai > 0:
            stock_buy_rq_liquid[stock] = min(stock_buy_liquid[stock],  int(v_rq_avai * p0)  + 1)
        else:
            stock_buy_rq_liquid[stock] = 0

    
         # 如果股票并不在多头头部，或者当日已经进行了空头开仓，则只能进行空头平仓，不能进行多头开仓
        if (score_array[stock] < (1 - params.open_params.pt.buy_prop_normal / 100.0)) or (stock in today_sell_stock_list):
            stock_buy_pt_liquid[stock] = 0 
        else:
            stock_buy_pt_liquid[stock] = max(stock_buy_liquid[stock] - stock_buy_rq_liquid[stock], 0)

        # 如果股票并不在空头头部，或者当日已经进行了多头开仓，则只能进行多头平仓，不能进行空头开仓
        if (score_array_sell[stock] > (params.open_params.rq.sell_prop_normal / 100.0)) or (stock in today_buy_stock_list):
            stock_sell_rq_liquid[stock] = 0
        else:
            if today_realtime_rq_dict.get(stock, 0) - v_rq_hold > 0:
                stock_sell_rq_liquid[stock] =  max(min(stock_sell_liquid[stock] - stock_sell_pt_liquid[stock], int((today_realtime_rq_dict.get(stock, 0) - v_rq_hold) * p0 + 1)),0)


        # 强制平仓条件：当天不可交易或者持仓周期过长，但如果该票当天本身就处于比较头部的位置，则可以不强制平仓
        # params.stock_expired_days记录了股票到期日，如果是股票多头，则默认是15天(避免同一只股票持仓周期过长)，如果是股票空头，则是实际融券到期日
        # 这里采用的方式是通过对打分进行调整，让组合优化自动识别到应该强制平仓这些股票
        if (stock not in params.today_istrade_stock_list) or (params.stock_expired_days.get(stock, 10) <= 0):
            if v_pt_hold > 0:
                if score_array[stock] < (1 - params.open_params.pt.buy_prop_normal / 100.0):
                    score_array_sell[stock] = 0
                    stock_sell_rq_liquid[stock] = 0
            if v_rq_hold > 0:
                score_array[stock] = 1
                stock_buy_pt_liquid[stock] = 0

    return pd.Series(stock_price_now), pd.Series(stock_buy_pt_liquid), pd.Series(stock_buy_rq_liquid), pd.Series(stock_sell_pt_liquid), pd.Series(stock_sell_rq_liquid),score_array,score_array_sell


In [ ]:
# 获取订单函数
def get_today_open_trade_task_ot(params):
    today_open_trade_task = []
    today_realtime_rq_dict = params.stock_realtime_rq_data_set[params.date] # 融券余量

    # 获取当前持仓股票的volume, 框架里面rq_volume全都是负数，-100代表融券100股
    stock_hold_pt = {code:round(stock_info.pt_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.pt_volume > 0}
    stock_available_pt = {code:round(stock_info.pt_available_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.pt_volume > 0}
    stock_hold_rq = {code:round(stock_info.rq_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.rq_volume < 0}
    stock_available_rq = {code:round(stock_info.rq_available_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.rq_volume < 0}

    # 当天股票+昨日股票取+可融并集
    stock_list = sorted(list(set(params.today_istrade_stock_list) | set(stock_hold_pt.keys()) | set(today_realtime_rq_dict.keys())| set(stock_hold_rq.keys())))
    # Barra_nmv_g、Barra_citic由我传入
    pre_date = get_shift_trade_date(params.date, -1)
    today_barra = read_local_data_daily("StockBarraData", pre_date)
    today_barra = today_barra.set_index("Code").reindex(stock_list)
    cmvg_mtx = pd.get_dummies(today_barra['Barra_nmv_g']).T.sort_index(axis=0).astype(int).fillna(0)
    cmvg_mtx = cmvg_mtx.values
    citic_mtx = pd.get_dummies(today_barra['Barra_citic']).T.sort_index(axis=0).astype(int).fillna(0)
    citic_mtx = citic_mtx.values


    # 获取买入打分，买入打分和纯多头策略相同
    model_score = params.today_daily_model_score.sort_values().dropna()
    score_array = (model_score.reindex(stock_list).rank(pct = True)).fillna(0.5)

    # 获取卖出打分，第一个版本先用买入打分，之后换成另一个打分
    model_score_sell = params.today_daily_model_score_sell.sort_values().dropna()
    score_array_sell = (model_score_sell.reindex(stock_list).rank(pct = True)).fillna(0.5)

    # 获取盘口流动性
    stock_price_now, stock_buy_pt_liquid, stock_buy_rq_liquid, stock_sell_pt_liquid, stock_sell_rq_liquid, score_array,score_array_sell = get_can_trade_amount_ot(stock_list,score_array, score_array_sell, params)
    stock_buy_pt_liquid= stock_buy_pt_liquid.reindex(stock_list).fillna(0)
    stock_buy_rq_liquid= stock_buy_rq_liquid.reindex(stock_list).fillna(0)
    stock_sell_pt_liquid= stock_sell_pt_liquid.reindex(stock_list).fillna(0)
    stock_sell_rq_liquid= stock_sell_rq_liquid.reindex(stock_list).fillna(0)
    stock_price_now = stock_price_now.reindex(stock_list).fillna(0)
    stock_hold_pt = pd.Series(stock_hold_pt).reindex(stock_list).fillna(0) # 当前持仓
    stock_hold_rq = pd.Series(stock_hold_rq).reindex(stock_list).fillna(0) # 当前持仓
    stock_rq = pd.Series(today_realtime_rq_dict).reindex(stock_list).fillna(0) # 当前可融余额
    stock_available_pt = pd.Series(stock_available_pt).reindex(stock_list).fillna(0)
    stock_available_rq = pd.Series(stock_available_rq).reindex(stock_list).fillna(0)

    # 对打分进行分布调整
    score_array = score_array - params.score_buy_adj
    score_array_sell = score_array_sell - params.score_sell_adj

    asset_base = params.pt_asset # 这个等于总可用保证金，多头可用保证金+多头已开保证金

    # 多头和空头共同约束
    n = len(stock_list)
    for i in range(0,30):
        cons = []
        stock_buy_pt = cp.Variable(n, nonneg = True)
        stock_sell_rq = cp.Variable(n, nonneg = True)
        stock_buy_rq = cp.Variable(n, nonneg = True)
        stock_sell_pt = cp.Variable(n, nonneg = True)

        k = cp.Variable(n, boolean=True) # 二元变量
        prehold_amt_pt = (stock_hold_pt * stock_price_now / asset_base).values
        prehold_amt_rq = (- stock_hold_rq * stock_price_now / asset_base).values

        allhold_amt_pt = stock_buy_pt - stock_sell_pt + prehold_amt_pt
        allhold_amt_rq = - stock_buy_rq + stock_sell_rq + prehold_amt_rq
        allhold_amt = allhold_amt_pt - allhold_amt_rq
        stock_buy = stock_buy_pt + stock_buy_rq
        stock_sell = stock_sell_pt + stock_sell_rq
        if prehold_amt_pt.sum() > 0: # 只要不是第一天开仓，就要有这个条件
            cons.extend([
                stock_buy <= k,
                stock_sell <= (1 - k),
            ])
        citic_share_limit, cmvg_share_limit = (i*0.2+1)*params.limit_params.citic_share_limit_stage1, (i*0.2+1)*params.limit_params.cmvg_share_limit_stage1
        
        # 市值约束
        all_cmvg = cmvg_mtx @ allhold_amt

        cons.extend([
            all_cmvg + cmvg_share_limit >= 0,
            all_cmvg - cmvg_share_limit <= 0,
        ])
        
        #行业约束
        all_citic = citic_mtx @  allhold_amt
        cons.extend([
                    all_citic + citic_share_limit >= 0 ,
                    all_citic - citic_share_limit <= 0,
                    ])
        
        # # 其他约束
        cons.extend([
            # 流动性限制
            stock_buy_pt <= stock_buy_pt_liquid / asset_base,
            stock_buy_rq <= stock_buy_rq_liquid / asset_base,
            stock_sell_pt <= stock_sell_pt_liquid / asset_base,
            stock_sell_rq <= stock_sell_rq_liquid / asset_base,

            # # 仓位限制
            cp.sum(allhold_amt)  <= 0,
            cp.sum(allhold_amt)  >= 0,

            cp.sum(allhold_amt_pt) <= 0.995,
            cp.sum(allhold_amt_pt) >= 0.985,

            # 换手率限制
            cp.sum(stock_buy_pt) <=  0.997 * max(params.open_params.pt.tvr_limit / 100.0, 1 - prehold_amt_pt.sum()), # 多头换手率
            cp.sum(stock_sell_rq) <=  0.997 * max(params.open_params.rq.tvr_limit / 100.0, 1 - prehold_amt_rq.sum()) , # 空头换手率

            # 融券持仓限制
            allhold_amt_pt <= np.maximum(params.open_params.pt.max_prop_total / 100.0, prehold_amt_pt),
            allhold_amt_rq <= np.maximum(params.open_params.rq.max_prop_total / 100.0, prehold_amt_rq),

        ])

        obj = cp.Maximize((score_array).values @ (stock_buy_pt *20 + stock_buy_rq) - (score_array_sell).values@(stock_sell_pt + stock_sell_rq *20) )
        prob = cp.Problem(obj, cons)
        # 求解
        ans = prob.solve(solver='SCIPY',verbose=False)
        if allhold_amt_pt.value is not None:
            logs = log_info(rf"第{i}轮有解")
            save_logs(params.logs_file, logs)
            break

    allhold_amt_pt1 = pd.Series(allhold_amt_pt.value)
    allhold_amt_rq1 = pd.Series(allhold_amt_rq.value)


    start_date = get_shift_trade_date(params.pre_date, -251)

    # params.daily_ret由我传入，为了满足dma相关性要求
    ret_ser = params.daily_ret.loc[start_date:params.pre_date].reindex(stock_list, axis = 1)
    ret_ser_centered = ret_ser.apply(lambda x:x.fillna(x.mean()), axis = 1).values
    T = 252
    n = len(stock_list)
    lscorr = pd.Series(ret_ser_centered @ allhold_amt_rq1).corr(pd.Series(ret_ser_centered @ allhold_amt_pt1))
    if lscorr < 0.83:
        logs = log_info(rf"多空相关性为{lscorr}，不满足要求，进一步求解")
        save_logs(params.logs_file, logs)
        for i in range(0,30):
            cons = []

            allhold_amt_pt = cp.Variable(n, nonneg = True)
            allhold_amt_rq = allhold_amt_rq1.values  # 在这一步中，allhold_amt_rq已经固定
            allhold_amt = allhold_amt_pt - allhold_amt_rq

            ret_centered_long = (ret_ser_centered @ allhold_amt_pt) - cp.sum(ret_ser_centered @ allhold_amt_pt) / T
            ret_centered_short = (ret_ser_centered @ allhold_amt_rq) - np.mean(ret_ser_centered @ allhold_amt_rq)
            # 协方差
            covariance = (1 / (T - 1)) * cp.sum(ret_centered_long * ret_centered_short)
            # 多头组合方差
            sigma_long = cp.norm(ret_centered_long) / np.sqrt(T - 1)
            # 空头组合方差
            sigma_short = np.sqrt(np.sum(ret_centered_short ** 2) / (T - 1))  

            cons.extend([
                covariance >= 0.85 * sigma_long * sigma_short,  # 相关性约束
            ])            

            citic_share_limit, cmvg_share_limit = (i*0.2+1)*params.limit_params.citic_share_limit_stage2, (i*0.2+1)*params.limit_params.cmvg_share_limit_stage2
            
            # 市值约束
            all_cmvg = cmvg_mtx @ (allhold_amt_pt - allhold_amt_rq)

            cons.extend([
                all_cmvg + cmvg_share_limit >= 0,
                all_cmvg - cmvg_share_limit <= 0,
            ])
            
            #行业约束
            all_citic = citic_mtx @  (allhold_amt_pt - allhold_amt_rq)
            cons.extend([
                        all_citic + citic_share_limit >= 0 ,
                        all_citic - citic_share_limit <= 0,
                        ])
            
            
            stock_change = (allhold_amt_pt - prehold_amt_pt)

            # 其他约束
            cons.extend([
                # 流动性限制
                cp.pos(stock_change) <= stock_buy_pt_liquid / asset_base,
                cp.neg(stock_change) <= stock_sell_pt_liquid / asset_base,

                # 仓位限制
                cp.sum(allhold_amt_pt - allhold_amt_rq)  <= 0,
                cp.sum(allhold_amt_pt - allhold_amt_rq)  >= 0,

                # 换手率限制
                cp.sum(cp.pos(allhold_amt_pt - prehold_amt_pt)) <=  0.997 * max(params.open_params.pt.tvr_limit / 100.0, 1 - prehold_amt_pt.sum()), # 多头换手率

                # 持仓限制
                allhold_amt_pt <= np.maximum(params.open_params.pt.max_prop_total / 100.0, prehold_amt_pt),

            ])
            obj = cp.Maximize((score_array).values @ (allhold_amt_pt))
            prob = cp.Problem(obj, cons)
            # 求解
            ans = prob.solve(verbose=False)
            if allhold_amt_pt.value is not None:
                logs = log_info(rf"第{i}轮有解")
                save_logs(params.logs_file, logs)
                break
    else:
        logs = log_info(rf"多空相关性为{lscorr}，满足要求，不需要进一步求解")
        save_logs(params.logs_file, logs)

    # 获取订单
    stock_order = pd.Series(((allhold_amt_pt - allhold_amt_rq) - (prehold_amt_pt - prehold_amt_rq)).value,index=stock_list)
    stock_order = stock_order[noteq(stock_order,0)].copy()
    stock_order = my_round2(stock_order * asset_base)
    stock_order = stock_order.sort_values()


    # 开始下单，这里由于买卖不在同一个是简单，和之前有点不一样
    today_open_trade_task = []
    today_can_ptbuy_amt = params.account.pt_cash # params.account.pt_cash意味着多头可开额度
    for stock, a_estimate in stock_order.iteritems():
        ## 风控和取整设定
        int_num = 1 if stock[:2] == '68' else 100 # 取整单位
        min_num = 200 if stock[:2] == '68' else 100 # 最小股数

        if a_estimate < 0: # 卖出阶段，有股票先卖股票，没股票再融券
            p_order = stock_price_now[stock]
            v_estimate = - a_estimate / p_order # 盘口挂单量

            # 先查询股票是否存在
            if stock in params.account.stock_infos.keys():
                v_hold = params.account.stock_infos[stock].pt_available_volume  # 查询普通户持仓
                if v_hold > 0:
                    open_trade_task = {}
                    # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
                    v_order = my_int(min(v_hold, v_estimate),  int_num)
                    if v_order > min_num: # 如果小于最小值没法卖
                        a_order = p_order * v_order
                        open_trade_task['date'] = params.date
                        open_trade_task['code'] = stock
                        open_trade_task['type'] = "PT"
                        open_trade_task['side'] = "S"
                        open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
                        open_trade_task['v_plan'] = v_order
                        open_trade_task['v_trans'] = 0
                        open_trade_task['a_trans'] = 0
                        open_trade_task['trade_type'] = "PT Sell"
                        open_trade_task['trade_time'] = params.open_params.pt.sell_time # 多头平仓算法单交易时间
                        open_trade_task['p0'] = stock_price_now[stock]
                        open_trade_task['priority'] = 1 # 回测需要，实盘不用理会
                        today_open_trade_task.append(open_trade_task) 
                        v_estimate -= v_order

            if stock_rq[stock] > 0: # 如果存在可融余额
                open_trade_task = {}
                # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
                v_order = my_int(min(stock_rq[stock], v_estimate),  int_num)
                if v_order > min_num: # 如果小于最小值没法卖
                    a_order = p_order * v_order
                    open_trade_task['date'] = params.date
                    open_trade_task['code'] = stock
                    open_trade_task['type'] = "RQ"
                    open_trade_task['side'] = "S"
                    open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
                    open_trade_task['v_plan'] = v_order
                    open_trade_task['v_trans'] = 0
                    open_trade_task['a_trans'] = 0
                    open_trade_task['trade_type'] = "RQ Sell"
                    open_trade_task['trade_time'] = params.open_params.rq.sell_time # 空头开仓算法单交易时间
                    open_trade_task['p0'] = stock_price_now[stock]
                    open_trade_task['priority'] = 4 # 回测需要，实盘不用理会
                    today_open_trade_task.append(open_trade_task)      

        elif a_estimate > 0: # 买入阶段
            p_order = stock_price_now[stock]
            v_estimate = a_estimate / p_order # 盘口挂单量

            if stock in params.account.stock_infos.keys():
                v_hold = - params.account.stock_infos[stock].rq_available_volume  # 查询普通户持仓
                if v_hold > 0:
                    open_trade_task = {}
                    # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
                    v_order = my_int(min(v_hold, v_estimate),  int_num)
                    if v_order > min_num: # 如果小于最小值没法卖
                        a_order = p_order * v_order
                        open_trade_task['date'] = params.date
                        open_trade_task['code'] = stock
                        open_trade_task['type'] = "RQ"
                        open_trade_task['side'] = "B"
                        open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
                        open_trade_task['v_plan'] = v_order
                        open_trade_task['v_trans'] = 0
                        open_trade_task['a_trans'] = 0
                        open_trade_task['trade_type'] = "RQ Buy"
                        open_trade_task['trade_time'] = params.open_params.rq.buy_time # 空头平仓算法单交易时间
                        open_trade_task['p0'] = stock_price_now[stock]
                        open_trade_task['priority'] = 2 # 回测需要，实盘不用理会
                        today_open_trade_task.append(open_trade_task) 
                    v_estimate -= v_order

            # 还券之后继续买入
            open_trade_task = {}
            # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
            v_order = my_int(min(v_estimate, today_can_ptbuy_amt / p_order) ,   int_num)
            if v_order > min_num: # 如果小于最小值没法卖
                a_order = p_order * v_order
                open_trade_task['date'] = params.date
                open_trade_task['code'] = stock
                open_trade_task['type'] = "PT"
                open_trade_task['side'] = "B"
                open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
                open_trade_task['v_plan'] = v_order
                open_trade_task['v_trans'] = 0
                open_trade_task['a_trans'] = 0
                open_trade_task['trade_type'] = "PT Buy"
                open_trade_task['trade_time'] = params.open_params.pt.buy_time # 多头开仓算法单交易时间
                open_trade_task['p0'] = stock_price_now[stock]
                open_trade_task['priority'] = 3 # 回测需要，实盘不用理会
                today_can_ptbuy_amt -= v_order * p_order * 1.01
                today_open_trade_task.append(open_trade_task)  

    today_open_trade_task = pd.DataFrame(today_open_trade_task) 
    today_open_trade_task["a_plan"] = today_open_trade_task["p0"] * today_open_trade_task["v_plan"] 
    today_open_trade_task = today_open_trade_task.sort_values("priority").reset_index(drop = True)  
    params.today_open_trade_task = today_open_trade_task # 当前情况只有一个交易时点
    logs = log_info(params.date, 'Open Task, Done! Stock Num=%d' % len(today_open_trade_task))
    save_logs(params.logs_file, logs)